In [1]:
import sys
sys.path.append('./..')
sys.path.append('./img2img')

import argparse
import os
# import setGPU
from src.datasets import get_dataset, DATASETS, get_num_classes, get_corrupted_dataset, load_parallel_model
from src.img2img.img2img_models import img2img_get_model, get_size
from src.core import Smooth, TSmooth
from time import time
# from src.grad2n import gradnorm_batch, gradnorm_batch_reduced
from collections import OrderedDict
from src.img2img.corruption import Corruption
# from src.analyze import plot_certified_accuracy, Line, ApproximateAccuracy
import torch
import datetime
from src.architectures import get_architecture
import numpy as np
from tqdm import tqdm

In [2]:
!nvidia-smi

Sun Nov 26 15:54:15 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           On  | 00000000:06:00.0 Off |                    0 |
| N/A   35C    P0              61W / 300W |  12487MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
sys.path.append('./../..')
from architectures import get_architecture

In [4]:
# smoothed_classifier = TSmooth(base_classifier, None, corruptor, get_num_classes(args.dataset),args.noise_dst,args.noise_sd, args.add_noise)

In [5]:
%env CUDA_VISIBLE_DEVICES=5
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_VISIBLE_DEVICES=5
env: CUDA_LAUNCH_BLOCKING=1


In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
parser = argparse.ArgumentParser(description='Certify many examples')
parser.add_argument("--dataset",
                    default='cifar10',choices=DATASETS, help="which dataset")
parser.add_argument("--base_classifier", type=str,

                    default=None,
                    help="path to saved pytorch model of base classifier")
# parser.add_argument("sigma", type=float, help="noise hyperparameter")
parser.add_argument("--outfile", type=str,
                    default='./logs/certify_results',
                    help="output file")


parser.add_argument("--batch", type=int, default=512, help="batch size")
parser.add_argument("--skip", type=int, default=30
                    , help="how many examples to skip")
parser.add_argument("--max", type=int, default=-1, help="stop after this many examples")
parser.add_argument("--split", choices=["train", "test"], default="test", help="train or test set")
parser.add_argument("--N0", type=int, default=100)
parser.add_argument("--N", type=int, default=1000, help="number of samples to use")
parser.add_argument("--alpha", type=float, default=1e-3, help="failure probability")





parser.add_argument('--corrupt',type=str,default=['none','gaussian_blur','motion_blur','zoom_blur','rotate','translate','contrast','pixelate','jpeg',][1],
                    help=' The corruption type for training')
parser.add_argument('--add_noise',type=float, default=0.0)
parser.add_argument('--noise_dst',default=["none","gaussian","exp","uniform","folded_gaussian"][2],type=str)
parser.add_argument('--noise_sd', default=0.8, type=float,
                    help="standard deviation of Gaussian noise for data augmentation")


parser.add_argument('--partial_min',default=0.0, type=float,
                    help = "Minimal of certify range")
parser.add_argument('--partial_max',default=1.0, type=float,
                    help = "Maximal of certify range")



parser.add_argument('--arch', type=str,
                    default=["edsr","unet","runet"][2])

args = parser.parse_args([])

In [8]:
args

Namespace(N=1000, N0=100, add_noise=0.0, alpha=0.001, arch='runet', base_classifier=None, batch=512, corrupt='gaussian_blur', dataset='cifar10', max=-1, noise_dst='exp', noise_sd=0.8, outfile='./logs/certify_results', partial_max=1.0, partial_min=0.0, skip=30, split='test')

In [9]:
from torch.utils.data import DataLoader, Dataset

def make_our_dataset_v2(data, idxes):
    images = []
    labels = []
    k = 0
    for i in tqdm(range(len(idxes))):
        image, label = data[idxes[i]]
        images.append(image.numpy())
        
        labels.append(label)

    images = np.array(images)
    labels = np.array(labels)
    dataset = CustomAudioDataset(images, labels)
    return dataset

class CustomAudioDataset(Dataset):

    def __init__(self, images, labels):
        self.images = torch.tensor(images)
        self.labels = torch.tensor(labels)
        
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        label =self.labels[idx]
        return image, label

In [10]:
np.random.seed(42)
NUM_IMAGES_FOR_TEST = 500
cifar10_ds = get_dataset('cifar10', 'test')
idxes2 = np.random.choice(len(cifar10_ds), NUM_IMAGES_FOR_TEST , replace=False)

our_cifar10_500 = make_our_dataset_v2(cifar10_ds, idxes2)
dataloader_cifar10 = DataLoader(our_cifar10_500, shuffle=False, batch_size=1, num_workers=6, pin_memory=False)

Files already downloaded and verified


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 5038.13it/s]


In [11]:
np.random.seed(42)

cifar100_ds = get_dataset('cifar100', 'test')
idxes3 = np.random.choice(len(cifar100_ds), NUM_IMAGES_FOR_TEST , replace=False)

our_cifar100_500 = make_our_dataset_v2(cifar100_ds, idxes3)
dataloader_cifar100 = DataLoader(our_cifar100_500, shuffle=False, batch_size=1,
                         num_workers=6, pin_memory=False)

Files already downloaded and verified


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 6077.49it/s]


In [12]:
idxes3

array([6252, 4684, 1731, 4742, 4521, 6340,  576, 5202, 6363,  439, 2750,
       7487, 5272, 5653, 3999, 6033,  582, 9930, 7051, 8158, 9896, 2249,
       4640, 9485, 4947, 9920, 1963, 8243, 6590, 8847,  321, 2678, 4625,
       4949, 8328, 3337, 5589,  251, 3973, 6630, 5547,   35, 8362, 1513,
       9317,   39, 4819, 3465, 1760, 2304, 3723, 8284, 4993, 8127, 3032,
       7938, 3039, 9655, 2545, 2592, 1188, 7966, 6077,  107, 1315, 8187,
       2753, 9753, 6231, 2876, 5323,  799, 3570, 2894, 2927, 8178,  971,
       6687, 8575, 2020, 9054,  952, 5359, 3857, 5861, 3145, 3305, 3006,
       9001, 7770, 7438, 7942, 9238, 1056, 3154, 3787, 9189, 7825, 7539,
       7231,  764, 5019, 4903, 8093, 6843, 2165, 1862, 7850, 9123, 2340,
       5527, 6743, 4003, 1123, 8709, 4049, 7640,  765, 2344, 4071, 7216,
       4216, 7952, 5970, 3350, 4695, 3399, 4081, 9149, 1224, 6292, 3122,
        510, 5794, 8181, 9417,   96, 6681, 1512, 8791,  119, 7791, 6517,
         33, 7887, 6830, 5421, 1068, 1347, 4747, 60

In [13]:
from datasets_utils import get_normalize_layer

In [14]:
from archs.cifar_resnet import resnet as resnet_cifar

def get_architecture(arch: str, dataset: str, device) -> torch.nn.Module:
    """ Return a neural network (with random weights)

    :param arch: the architecture - should be in the ARCHITECTURES list above
    :param dataset: the dataset - should be in the datasets.DATASETS list
    :return: a Pytorch module
    """
    if arch == "resnet50" and dataset == "imagenet":
#         model = torch.nn.DataParallel(resnet50(pretrained=False)).cuda()
#         cudnn.benchmark = True
        model = resnet50(pretrained=False).to(device)
    elif arch == "cifar_resnet20":
        model = resnet_cifar(depth=20, num_classes=10).to(device)
    elif arch == "cifar_resnet110":
        model = resnet_cifar(depth=110, num_classes=10).to(device)
    elif arch == "cifar100_resnet110":
        model = resnet_cifar(depth=110, num_classes=100).to(device)
    elif arch == "fashion_22full":
        model = Conv2FC2full()
        model = model.to(device)
    elif arch == "fashion_22simple":
        model = Conv2FC2simple().to(device)
    elif arch == "mnist_43":
        model = Conv4FC3().to(device)
    normalize_layer = get_normalize_layer(dataset, device=device)
    return torch.nn.Sequential(normalize_layer, model)


# Contrast
# or is it Contrast-Brightness(??) according to the code

In [15]:
args.corrupt = 'contrast'
args.noise_sd = 0.3
args.noise_dst = "gaussian"

## CIFAR-10

In [16]:
args.base_classifier = "../../tss_weights/cifar10/resnet110/brightness-contrast/0.3_0.3/consistency/checkpoint.pth.tar"
# args.base_classifier = "../../new_results/cifar100_resnet110_cifar100_cb_cifar100_cb_checkpoint.pth.tar"

args.dataset = 'cifar10'

dataset = our_cifar10_500



device = torch.device('cuda')
checkpoint = torch.load(args.base_classifier, map_location=device)
base_classifier = get_architecture(checkpoint["arch"], args.dataset, device=device)
base_classifier.load_state_dict(checkpoint["state_dict"])


img_size, param_size = get_size(args)

model_t = img2img_get_model(args, img_size, param_size,parallel=False)

trans_path = None
if trans_path is not None:
    state_dict = torch.load(trans_path)['state_dict']
    model_t.load_state_dict(state_dict)
# model_t = load_parallel_model(model_t, state_dict)

# to device
base_classifier.to(device)
base_classifier.eval()
model_t.to(device)
model_t.eval()


#load dataset

# corruptor
corruptor = Corruption(args, co_type=args.corrupt,add_noise=args.add_noise,noise_sd=args.noise_sd,distribution=args.noise_dst)

# create the smooothed classifier g
smoothed_classifier = TSmooth(base_classifier, None, corruptor, get_num_classes(args.dataset),args.noise_dst,args.noise_sd, args.add_noise)

# prepare output file
f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime", file=f, flush=True)
print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime")

tot, tot_good, tot_correct = 0, 0, 0

# for gaussian smooth
attack_radius = args.partial_max


# dataset = get_dataset(args.dataset, args.split)
dataset = our_cifar10_500
for i in tqdm(range(len(dataset))):

#     # only certify every args.skip examples, and stop after args.max examples
#     if i % args.skip != 0:
#         continue
#     if i == args.max:
#         break

    (x, label) = dataset[i]

    before_time = time()
    x = x.cuda()
    prediction, radius = smoothed_classifier.certify(x, args.N0, args.N, args.alpha, args.batch)


    correct = (prediction == label).item()
    cond1 = radius * args.noise_sd > args.partial_max
#     print(cond1.dtype)
#     print(correct.dtype)
    good = (radius * args.noise_sd > args.partial_max)&correct

    tot, tot_good, tot_correct = tot+1, tot_good+good, tot_correct+correct
    after_time = time()
    time_elapsed = str(datetime.timedelta(seconds=(after_time - before_time)))
    print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed))

f.close()

print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot))

f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot), file=f, flush=True)
f.close()

idx	label	predict	radius	good	correct	time


  0%|▎                                                                                                                                        | 1/500 [00:04<36:58,  4.45s/it]

0	2	2	1.60457	False	True	0:00:04.446386


  0%|▌                                                                                                                                        | 2/500 [00:08<35:46,  4.31s/it]

1	1	1	2.46326	False	True	0:00:04.212332


...

497	1	1	2.46326	False	True	0:00:03.712667


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 499/500 [34:36<00:03,  3.63s/it]

498	0	0	2.46326	False	True	0:00:03.792025


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [34:39<00:00,  4.16s/it]

499	8	8	2.46326	False	True	0:00:03.459442
Total 500 Certified 0 Certified Acc 0.0 Test Acc 0.86


## CIFAR-100

In [17]:
# args.base_classifier = "../../tss_weights/cifar10/resnet110/brightness-contrast/0.3_0.3/consistency/checkpoint.pth.tar"
args.base_classifier = "../../new_results/cifar100_resnet110_cifar100_cb_cifar100_cb_checkpoint.pth.tar"
args.corrupt = 'contrast'
args.dataset = 'cifar100'
args.noise_sd = 0.3
dataset = our_cifar100_500



device = torch.device('cuda')
# checkpoint = torch.load(args.base_classifier, map_location=device)
# base_classifier = get_architecture(checkpoint["arch"], args.dataset)
# base_classifier.load_state_dict(checkpoint["state_dict"])


model = get_architecture(arch="cifar100_resnet110", dataset="cifar100", device=device)
checkpoint = torch.load(args.base_classifier, map_location=device)
model.load_state_dict(checkpoint['state_dict'])
model.eval()
model = model.to(device)

base_classifier = model





# load the base classifier

# args.base_classifier = "../../tss_weights/cifar10/resnet110/brightness-contrast/0.4_0.4/consistency/checkpoint.pth.tar"





img_size, param_size = get_size(args)

model_t = img2img_get_model(args, img_size, param_size,parallel=False)

trans_path = None
if trans_path is not None:
    state_dict = torch.load(trans_path)['state_dict']
    model_t.load_state_dict(state_dict)
# model_t = load_parallel_model(model_t, state_dict)

# to device
base_classifier.to(device)
base_classifier.eval()
model_t.to(device)
model_t.eval()


#load dataset

# corruptor
corruptor = Corruption(args, co_type=args.corrupt,add_noise=args.add_noise,noise_sd=args.noise_sd,distribution=args.noise_dst)

# create the smooothed classifier g
smoothed_classifier = TSmooth(base_classifier, None, corruptor, get_num_classes(args.dataset),args.noise_dst,args.noise_sd, args.add_noise)

# prepare output file
f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime", file=f, flush=True)
print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime")

tot, tot_good, tot_correct = 0, 0, 0

# for gaussian smooth
attack_radius = args.partial_max




for i in tqdm(range(len(dataset))):

#     # only certify every args.skip examples, and stop after args.max examples
#     if i % args.skip != 0:
#         continue
#     if i == args.max:
#         break

    (x, label) = dataset[i]

    before_time = time()
    x = x.cuda()
    prediction, radius = smoothed_classifier.certify(x, args.N0, args.N, args.alpha, args.batch)


    correct = (prediction == label).item()
    cond1 = radius * args.noise_sd > args.partial_max
#     print(cond1.dtype)
#     print(correct.dtype)
    good = (radius * args.noise_sd > args.partial_max)&correct

    tot, tot_good, tot_correct = tot+1, tot_good+good, tot_correct+correct
    after_time = time()
    time_elapsed = str(datetime.timedelta(seconds=(after_time - before_time)))
    print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed))

f.close()

print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot))

f = open(args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot), file=f, flush=True)
f.close()

idx	label	predict	radius	good	correct	time


  0%|▎                                                                                                                                        | 1/500 [00:03<31:03,  3.73s/it]

0	10	10	0.83511	False	True	0:00:03.734061


  0%|▌                                                                                                                                        | 2/500 [00:07<31:09,  3.75s/it]

1	46	-1	0.00000	False	False	0:00:03.764977


...

497	67	67	2.46326	False	True	0:00:03.274933


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 499/500 [29:46<00:03,  3.39s/it]

498	74	78	2.46326	False	False	0:00:03.598729


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [29:49<00:00,  3.58s/it]

499	29	95	0.54053	False	False	0:00:03.598776
Total 500 Certified 0 Certified Acc 0.0 Test Acc 0.47


In [11]:
2

2

In [12]:
## Gsmooth

print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot))

Total 500 Certified 413 Certified Acc 0.826 Test Acc 0.866


In [15]:


f = open(args.outfile+'_'+args.corrupt+'_'+str(args.noise_sd), 'w')
print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot), file=f, flush=True)
f.close()